In [22]:
import tensorflow as tf
tf.keras.utils.disable_interactive_logging() # Disabling interactive logging to avoid cluttering the output

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# 1. Data Preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

# Training generator (80% of data)
train_gen = train_datagen.flow_from_directory(
    '../dataset',  # Your main dataset folder
    target_size=(228, 228),  # Changed to (128, 128) for consistency
    batch_size=32,
    class_mode='categorical',
    subset='training',  # Uses 80% of images
    shuffle=True
)

# Validation generator (20% of data)
val_gen = train_datagen.flow_from_directory(
    '../dataset',  # Same main folder
    target_size=(228, 228),  # Changed to (128, 128) for consistency
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # Uses 20% of images
    shuffle=False
)

# 2. Model Architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(228, 228, 3)), # Input shape adjusted to (128, 128, 3)
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # 4 output classes
])

# 3. Model Compilation
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. Callbacks (Important!)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('../models/eye_disease_model.keras', save_best_only=True)
]

# 5. Training
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,  # Start with 30 epochs
    callbacks=callbacks
)

# 6. Save the final model
model.save('../models/final_texture_classifier.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Load the best saved model
model = load_model('../models/eye_disease_model.keras')

# Reduce learning rate for finer adjustments
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training
history_continued = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,  # Additional epochs
    callbacks=[EarlyStopping(patience=3)]  # New callback
)

# Save the continued model
model.save('../models/refined_model.keras')

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.legend()
plt.savefig('training_history.png')

In [ ]:
print("Final Training Accuracy: {:.2f}%".format(history.history['accuracy'][-1] * 100))
print("Final Validation Accuracy: {:.2f}%".format(history.history['val_accuracy'][-1] * 100))